<a href="https://colab.research.google.com/github/fursew05/DOE/blob/main/%EC%8B%A4%ED%97%98%EA%B3%84%ED%9A%8D%EB%B2%95_2%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2장 Experiment with a Single Factor

## CRD(Completely Randomized Design)
- 두 집단의 평균을 비교하기 위해서 CRD를 가정해야함

- CRD란?
실험단위들의 각 처리 그룹 배치와 실험 순서를 랜덤으로 정하여 실험을 설계하는 방법을 말한다.

  $y_{ij} = \mu + \tau_i + \varepsilon_{ij}, \quad \sum_{i=1}^{t}\tau_i=0$

## 일원배치 분산분석
CRD에서 고려하는 요인이 한가지인 경우에 사용하는 분산분석 방법이다.
예를 들어 어떤 모터의 수명을 종속변수로 두었을때 배터리의 종류가 2가지인 경우에는 t-test를 사용하면 되지만, 종류가 3가지 즉 3개 이상의
집단의 평균을 비교하는 경우 분산분석을 진행하게 된다.

이때 실험하게 되는 횟수는 요인의 level 수 * 샘플의 수가 된다
배터리 종류 3개에 대해서 모터의 수명시간을 10번씩 측정한다면 30회 실험 진행

In [7]:
import pandas as pd

df = pd.read_csv('/content/sample_data/mtcars.csv',index_col=0)
df.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [2]:
# t-test : gear가 3과 4인 집단의 disp 평균을 비교하고 싶은 경우 ttest_ind 함수 활용
from scipy.stats import ttest_ind
gear_3 = df[df['gear']==3]['disp']
gear_4 = df[df['gear']==4]['disp']

result = ttest_ind(gear_3,gear_4)
print(f"t통계량 : {result[0]}")
print(f"p값 : {result[1]}")

t통계량 : 6.949417317275277
p값 : 2.767535093193568e-07


유의수준 5%에서 귀무가설을 기각할 수 있어 gear_3와 gear_4의
평균의 차이가 있다는 것을 알 수 있음

- 귀무가설 : 집단 간 평균의 차이가 없다
- 대립가설 : 집단 간 평균의 차이가 있다

- 일원분산분석을 하는 경우 다음의 사전검정이 필요함


1.   정규성 확인 : 변수의 왜도 값이 절댓값 2를 넘지 않는지 확인
2.   등분산 검정(bartlett test 또는 levene test) : H0는 변수 간 분산의 차이가 없다


In [3]:
# One way ANOVA : gear가 3,4,5 세 집단의 disp 평균을 비교하고 싶은 경우
from scipy.stats import skew, bartlett

# 정규성 확인
gear_5 = df[df['gear']==5]['disp']

normality_result_gear_3 = skew(gear_3)
normality_result_gear_4 = skew(gear_4)
normality_result_gear_5 = skew(gear_5)

print(f"gear_3의 왜도 : {normality_result_gear_3}")
print(f"gear_4의 왜도 : {normality_result_gear_4}")
print(f"gear_5의 왜도 : {normality_result_gear_5}")

#등분산성 확인
bartlett_result = bartlett(gear_3,gear_4,gear_5)
print(f"등분산 검정의 p값 : {bartlett_result[1]}")

gear_3의 왜도 : -0.2668791892901188
gear_4의 왜도 : -0.2003117299768987
gear_5의 왜도 : 0.40812918060558423
등분산 검정의 p값 : 0.010299628043890017


In [4]:
# 일원배치 분산분석 : F 통계량을 통하여 검정진행
# 귀무가설 : 기어 개수에 따른 배기량 평균의 차이가 없다
# 대립가설 : 기어 개수에 따른 배기량 평균의 차이가 존재한다.

# 방법 1 : f_oneway
from scipy.stats import f_oneway
result = f_oneway(gear_3,gear_4,gear_5)
if result[1] < 0.05:
  print("귀무가설 기각")
else:
  print("귀무가설을 기각할 수 없음")

귀무가설 기각


In [12]:
# 방법 2: ols 활용하여 anova 테이블 만들기
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# ols 함수 안에는 '종속변수 이름 ~ 설명변수 이름', 데이터프레임 두가지를 넣으면 됨
# 설명변수가 범주형일때에는 C(변수명) 형식으로 넣어야 함
model = ols('disp ~ C(gear)', df).fit()
anova_result = anova_lm(model)

print(anova_result)
if anova_result['PR(>F)'].iloc[0] < 0.05:
  print("귀무가설 기각")
else:
  print("귀무가설을 기각할 수 없음")

            df         sum_sq        mean_sq          F    PR(>F)
C(gear)    2.0  280220.630021  140110.315010  20.734399  0.000003
Residual  29.0  195964.164667    6757.384989        NaN       NaN
귀무가설 기각


- p값이 정해놓은 유의수준보다 작은 경우 귀무가설을 기각할 수 있게 됨

  따라서 gear 별로 disp의 평균에 통계적으로 유의미한 차이가 있다는 것을 알 수 있음

## Contrasts
위에서 진행한 방법은 일원배치 분산분석 중 global test로 한가지 요인에 대해서 여러 집단의 평균 차이가 있다는 것은 알 수 있지만 차이가 어디서 나타나는지는 알 수가 없음.

Contrasts는 평균 벡터와 선형결합을 하게되는 벡터를 말한다

$L = \sum_{i=1}^{k} c_i \mu_i, \quad \text{단, } \sum_{i=1}^{k} c_i = 0$

예시) 위에 예시에서 global F test를 진행했을때 차이가 있다는 것을 확인했지만
어떤 집단에서 차이를 보이는지 알기위해서 기어가 3인 집단과 기어가 5인 집단의
차이가 존재하는지 확인하고 싶음

이때 벡터 C (1,0,-1)과 평균 벡터 (mu_3,mu_4,mu_5)간의 선형결합 값을 모수로 두어 가설 검정을 진행

- 귀무가설 : 선형결합이 0이다
- 대립가설 : 선형결합이 0이 아니다


In [11]:
# mu3-mu5 = 0인지 확인하는 예제
from statsmodels.stats.contrast import Contrast
import numpy as np

model = ols('disp ~ C(gear)', data = df).fit()
contrast_vector = np.array([1,0,-1])
model.t_test(contrast_vector)

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0           450.1200     56.156      8.016      0.000     335.269     564.971

- p값이 매우 작은것으로 보아 기어3 집단과 기어5 집단 간 차이가 있다는 것을  확인할 수 있음

In [ ]:
# orthogonal contrast